In [15]:
from lxml import html
import requests
import bs4 as BeautifulSoup
import pandas as pd
from datetime import date

In [39]:
url = "https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001"
page = requests.get(url)
tree = str(page.content)
soup = BeautifulSoup.BeautifulSoup(tree, "html")


In [34]:
def url_from_string(date:'string'):
    # date must be string in the format "YYYY-MM-DD"
    year = date[0:4] 
    month = date[5:7]
    month = int(month)
    month = str(month-1) # because month index starts at 0
    day = date[8:10] 
    if day[0] == "0" :
        day = day[1:] # in the url single digit days are single digits
    url = "https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001&jour2="+ day + "&mois2="+month+"&annee2="+year
    return url 

'https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001&jour2=31&mois2=9&annee2=2023'

In [37]:
## Extraction de toutes les dates
origin = pd.Timestamp('2019-09-01') #premiere date où les releves sont disponibles
dti = pd.date_range(origin, date.today(), freq="d") # recuperation de toutes les dates jusqu'à aujourd'hui

for index in range(len(dti)):
    date_string = str(dti[index])
    url = url_from_string(date_string)
    print(url)



https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001&jour2=1&mois2=8&annee2=2019
https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001&jour2=2&mois2=8&annee2=2019
https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001&jour2=3&mois2=8&annee2=2019
https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001&jour2=4&mois2=8&annee2=2019
https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001&jour2=5&mois2=8&annee2=2019
https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001&jour2=6&mois2=8&annee2=2019
https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001&jour2=7&mois2=8&annee2=2019
https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001&jour2=8&mois2=8&annee2=2019
https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001&jour2=9&mois2=8&annee2=2019
https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63282001&jour2=10&mois2=8&annee2=2019
https://www.meteociel.fr/temps-reel/obs_villes.php?code2=63

In [40]:
print(soup.prettify())

<html>
 <body>
  <p>
   b'
   <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
   \n
  </p>
  \n\n
  <link href="//static.meteociel.fr/css/css.css" rel="stylesheet" type="text/css"/>
  \n
  <title>
   Meteociel - Observations Plauzat (63) - donn\xe9es m\xe9t\xe9o de la station  - Tableaux horaires en temps réel
  </title>
  \n
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  \n
  <meta content="Relev\xe9s horaires des observations m\xe9t\xe9o de Plauzat (63). Temp\xe9ratures, pression, pr\xe9cipitations, vent en temps r\xe9el " name="description"/>
  \n\n
  <meta content="meteo, meteociel, temps r\xe9el, temps reel, observations, observation, meteo ciel, meteociel, pluie, precipitations, soleil, temp\xe9rature, temp, neige, report, rapport, donn\xe9es, relev\xe9, radars, satallite, meteo" lang="fr" name="keywords"/>
  \n
  <meta content="fr" http-equiv="Content-Language"/>
  \n
  <meta content="webmaster@mete

In [103]:
var = soup.find(string='Heure').parent.parent.parent ## Ligne de header sur le site

print(var.prettify())


<tr>
 <td align="center" width="6%">
  <b>
   Heure
   <br/>
   locale
  </b>
 </td>
 <td align="center">
  <b>
   Visi
  </b>
 </td>
 <td width="10%">
  <div align="center">
   <b>
    Temp\xe9rature
   </b>
  </div>
 </td>
 <td>
  <div align="center">
   <b>
    Humi.
   </b>
  </div>
 </td>
 <td width="8%">
  <div align="center">
   <b>
    Point de rosée
   </b>
  </div>
 </td>
 <td align="center">
  <b>
   Humidex
  </b>
 </td>
 <td align="center">
  <b>
   Windchill
  </b>
 </td>
 <td colspan="2">
  <div align="center">
   <b>
    Vent (rafales)
   </b>
  </div>
 </td>
 <td width="11%">
  <div align="center">
   <b>
    Pression
   </b>
  </div>
 </td>
 <td width="8%">
  <div align="center">
   <b>
    Pr\xe9cip.
    <br/>
    mm/h
   </b>
  </div>
 </td>
</tr>



In [46]:
##### On cherche à scrapper le tableau et le mettre dans un pd.df

# On récupère la balise englobant tout le tableau
var = soup.find(string='locale').parent.parent.parent.parent

# Définition du df

df = pd.DataFrame(columns=['Time', 'Temperature', 'Precipitations'])
list = []

# Collecting data
for row in var.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    if(columns != []):
        max_col_index = len(columns)-1
        time = columns[0].text.strip()
        temperature = columns[2].text.strip()
        precipitations = columns[max_col_index].text.strip() #.span.contents[0].strip('&0.')
        list.append([time, temperature, precipitations])
        #newdf = pd.DataFrame({'Time': time,  'Temperature': temperature, 'Precipitations': precipitations}, ignore_index=True)
        
df = pd.DataFrame(list)       

In [42]:
df

,0,1,2
0,Heurelocale,Temp\xe9rature,Pr\xe9cip. mm/h
1,15 h,34.2 \xb0C,aucune
2,14 h,33.3 \xb0C,aucune
3,13 h,30.7 \xb0C,aucune
4,12 h,28.2 \xb0C,aucune
5,11 h,25 \xb0C,aucune
6,10 h,21.2 \xb0C,aucune
7,9 h,19.3 \xb0C,aucune
8,8 h,17 \xb0C,aucune
9,7 h,16.7 \xb0C,aucune


In [43]:
# Corrections à retirer quand la fonction du dessurs sera plus propre
df = pd.DataFrame(list)       
df = df[1:] # On ôte la première ligne 
df.rename(columns={0:"heure", 1:"temperature", 2:"precipitations"}, inplace=True) # renommer première ligne

df["temperature"]= df["temperature"].str[:-6] # on retire les horribles caracteres de degres
df["heure"]=df["heure"].str[:-2] # on retire le caractère h 
df["precipitations"] = df["precipitations"].replace("aucune", "0 mm")
print(df)
df["precipitations"] = df["precipitations"].str[:-2]
df.head()





   heure temperature precipitations
1     15        34.2           0 mm
2     14        33.3           0 mm
3     13        30.7           0 mm
4     12        28.2           0 mm
5     11          25           0 mm
6     10        21.2           0 mm
7      9        19.3           0 mm
8      8          17           0 mm
9      7        16.7           0 mm
10     6        16.8           0 mm
11     5        17.2           0 mm
12     4        18.1           0 mm
13     3        18.9           0 mm
14     2        19.2           0 mm
15     1        19.8           0 mm
16     0        20.5           0 mm


,heure,temperature,precipitations
1,15,34.2,0
2,14,33.3,0
3,13,30.7,0
4,12,28.2,0
5,11,25,0


,heure,temperature,precipitations
1,15,34.2,0
2,14,33.3,0
3,13,30.7,0
4,12,28.2,0
5,11,25,0
6,10,21.2,0
7,9,19.3,0
8,8,17,0
9,7,16.7,0
10,6,16.8,0


In [48]:
### fonction nettoyage
def nettoyage(data):
    result = data[1:] # On ôte la première ligne, a changer quand ce sera plus correct
    result.rename(columns={0:"heure", 1:"temperature", 2:"precipitations"}, inplace=True) # renommer première ligne

    result.loc["temperature"]= result.loc["temperature"].str[:-6] # on retire les horribles caracteres de degres
    result.loc["heure"]=result.loc["heure"].str[:-2] # on retire le caractère h 
    result.loc["precipitations"] = result.loc["precipitations"].replace("aucune", "0 mm")
    result.loc["precipitations"] = result.loc["precipitations"].str[:-2]



In [172]:
[var = soup.find(string='locale').parent.parent.parent.parent]

In [49]:
var = nettoyage(df)
print(var)

/var/folders/x9/h8vwd75d3352d6ngzx4sqgj00000gn/T/ipykernel_867/2373855069.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.rename(columns={0:"heure", 1:"temperature", 2:"precipitations"}, inplace=True) # renommer première ligne


KeyError: 'temperature'

In [ ]:
### Fonction travaillant sur le texte pour remplacer les balises html par des balises CSV
def line_to_csv(variable):
    result = variable
    result = result.replace('<div align="center">', "")
    result = result.replace('</div>', '')
    result = result.replace('<td>', '')
    result = result.replace("</td>", ",")
    result = result.replace('<td align="center">', "")
    result = result.replace("°C", "")
    result = result.replace('<tr>', '')
    result = result.replace("</tr>", "\n")
    return result


' 15 h, \xa0, 27.4 \\xb0C, \xa0 , ,  , \xa0, \xa0, \xa0, \xa0 ,  aucune, \n'

ImportError: attempted relative import with no known parent package